##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 4 - Fine tuning a custom model

Welcome back to the Kaggle 5-day Generative AI course!

In this notebook you will use the Gemini API to fine-tune a custom, task-specific model. Fine-tuning can be used for a variety of tasks from classic NLP problems like entity extraction or summarisation, to creative tasks like stylised generation. You will fine-tune a model to classify the category a piece of text (a newsgroup post) into the category it belongs to (the newsgroup name).

This codelab walks you tuning a model with the API. [AI Studio](https://aistudio.google.com/app/tune) also supports creating new tuned models directly in the web UI, allowing you to quickly create and monitor models using data from Google Sheets, Drive or your own files.

In [21]:
%pip install  google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [26]:
import google.generativeai as genai



### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [27]:
import os
from dotenv import load_dotenv
   
   
load_dotenv()
GOOGLE_API_KEY = os.getenv('VERTEXAI_API_KEY')

In [28]:
#from kaggle_secrets import UserSecretsClient

#GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Explore available models

You will be using the [`TunedModel.create`](https://ai.google.dev/api/tuning#method:-tunedmodels.create) API method to start the fine-tuning job and create your custom model. Find a model that supports it through the [`models.list`](https://ai.google.dev/api/models#method:-models.list) endpoint. You can also find more information about tuning models in [the model tuning docs](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?lang=python).

In [60]:
try:
    models_generator = genai.list_models()
    
    # Convert the generator to a list to print all models
    models = list(models_generator)
    
    # Print each model
    for model in models:
        print(model)
except Exception as e:
    print(f"An error occurred while listing models: {e}")


Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko

In [64]:
# Assuming you have already imported the necessary module and initialized the model
import google.generativeai as genai

# Initialize the model (replace with actual model name if needed)
model_name = 'models/text-bison-001'
model = genai.GenerativeModel(model_name)

# Use dir() to list all attributes and methods
#print(dir(model))
#help(model.generate_content)
response = model.generate_content(prompt="What are the symptoms of the common cold?")
print(response.text)

TypeError: generate_content() got an unexpected keyword argument 'prompt'

In [38]:
!pip3 install --upgrade --user google-cloud-aiplatform
from vertexai.preview.language_models import TextGenerationModel

   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
    --------------------------------------- 0.2/6.2 MB 4.5 MB/s eta 0:00:02
   -- ------------------------------------- 0.3/6.2 MB 4.1 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/6.2 MB 3.5 MB/s eta 0:00:02
   ---- ----------------------------------- 0.6/6.2 MB 4.0 MB/s eta 0:00:02
   ----- ---------------------------------- 0.8/6.2 MB 3.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.0/6.2 MB 3.7 MB/s eta 0:00:02
   ------- -------------------------------- 1.2/6.2 MB 3.8 MB/s eta 0:00:02
   -------- ------------------------------- 1.4/6.2 MB 3.8 MB/s eta 0:00:02
   --------- ------------------------------ 1.5/6.2 MB 3.7 MB/s eta 0:00:02
   ---------- ----------------------------- 1.7/6.2 MB 3.6 MB/s eta 0:00:02
   ------------ --------------------------- 1.9/6.2 MB 3.6 MB/s eta 0:00:02
   ------------- -------------------------- 2.0/6.2 MB 3.6 MB/s eta 0:00:02
   -------------- -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


ModuleNotFoundError: No module named 'vertexai'

In [ ]:


def interview(temperature: float = .2):
    """Ideation example with a Large Language Model"""

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 256,
        "top_p": .8,
        "top_k": 40,
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        'Give me ten interview questions for the role of program manager.',
        **parameters,
    )
    print(f"Response from Model: {response.text}")